In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

import warnings
warnings.filterwarnings('ignore')

In [2]:
era5_features = [
        # Temperature variables
        'temperature_2m',                       # Air temperature
        'temperature_2m_min',                   # Daily minimum air temperature
        'temperature_2m_max',                   # Daily maximum air temperature
        'soil_temperature_level_1',             # Topsoil temperature (0-7 cm)
        'soil_temperature_level_2',             # Soil temperature (7-28 cm)
        
        # Moisture variables
        'volumetric_soil_water_layer_1',        # Topsoil moisture content
        'volumetric_soil_water_layer_2',        # Soil moisture (7-28 cm)
        'volumetric_soil_water_layer_3',        # Soil moisture (28-100 cm)
        'total_precipitation_sum',              # Total rainfall and snow
        'dewpoint_temperature_2m',              # Air humidity indicator
        
        # Radiation and energy variables
        'surface_solar_radiation_downwards_sum', # Solar radiation at surface
        'surface_net_solar_radiation_sum',       # Net solar radiation at surface
        
        # Evaporation and water cycle
        'total_evaporation_sum',                 # Actual evaporation
        
        # Wind variables
        'u_component_of_wind_10m',               # East-west wind component
        'v_component_of_wind_10m'                # North-south wind component
    ]

openweather_features = [
        'temp',         # Current temperature (C or K depending on units)
        'feels_like',   # Perceived temperature considering humidity and wind (C or K)
        'temp_min',     # Minimum temperature at the moment (C or K)
        'temp_max',     # Maximum temperature at the moment (C or K)
        'pressure',     # Atmospheric pressure at sea level (hPa)
        'humidity',     # Humidity percentage (%)
        'wind_speed',   # Wind speed (meter/sec)
        'wind_deg',     # Wind direction in degrees (0–360)
        'rain_1h',      # Rain volume for the last 1 hour (mm)
        'rain_3h',      # Rain volume for the last 3 hours (mm)
        'clouds_all'    # Cloudiness percentage (%)
    ]

chirts_features = [
        'heat_index',
        'maximum_temperature',
        'minimum_temperature',
        'relative_humidity',
        'saturation_vapor_pressure',
        'vapor_pressure_deficit',
    ]

power_features = [
        'T2M',           # MERRA-2 Temperature at 2 Meters (C)
        'T2MDEW',        # MERRA-2 Dew/Frost Point at 2 Meters (C)
        'T2MWET',        # MERRA-2 Wet Bulb Temperature at 2 Meters (C)
        'TS',            # MERRA-2 Earth Skin Temperature (C)
        'T2M_RANGE',     # MERRA-2 Temperature at 2 Meters Range (C)
        'T2M_MAX',       # MERRA-2 Temperature at 2 Meters Maximum (C)
        'T2M_MIN',       # MERRA-2 Temperature at 2 Meters Minimum (C)
        'PS',            # MERRA-2 Surface Pressure (kPa)
        'WS2M',          # MERRA-2 Wind Speed at 2 Meters (m/s)
        'WS2M_MAX',      # MERRA-2 Wind Speed at 2 Meters Maximum (m/s)
        'WS2M_MIN',      # MERRA-2 Wind Speed at 2 Meters Minimum (m/s)
        'GWETTOP',       # MERRA-2 Surface Soil Wetness (1)
        'GWETROOT'      # MERRA-2 Root Zone Soil Wetness (1)
    ]

selected_features = era5_features

In [3]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# konfigurasi parameter
start_year = 2020
end_year = 2025
climate_dir = "climate_timeseries/cleaned/"
vegetation_dir = "pandanwangi_timeseries/"
viz_dir = "visualization/"
evi = True
kecamatan_names = ['warungkondang']

era5_datasets = ['era5_warungkondang.csv']
chirts_datasets = ['chirts_warungkondang.csv']
ow_datasets = ['OpenWeather_warungkondang.csv']
power_datasets = ['power_warungkondang.csv']

selected_dataset = era5_datasets

available_vegetation_datasets = ['warungkondang.csv']

selected_kec = kecamatan_names[0]

if evi:
    available_vegetation_datasets = ['evi_' + dataset for dataset in available_vegetation_datasets]

extract_statistics = ['mean', 'min', 'max', 'std']
statistic_corr = ['mean', 'min', 'max', 'std', 'smoothed_mean']

temporal_resolution = '5D'


In [9]:
# load seluruh data climate
def load_climate_data(climate_dir, datasets, kecamatan_names):
    climate_df = {}
    for i, dataset in enumerate(selected_dataset):
        try:
            curr = pd.read_csv(f"{climate_dir}/{dataset}")
            kec = kecamatan_names[min(i, len(kecamatan_names)-1)]
            climate_df[kec] = curr
        except Exception as e:
            print(f"File kecamatan {kec} tidak dapat dimuat: {e}")
    return climate_df

# load seluruh data vegetasi
def load_vegetation_data(vegetation_dir, datasets, kecamatan_names):
    vegetation_df = {}
    for i, dataset in enumerate(datasets):
        try:
            curr = pd.read_csv(f"{vegetation_dir}/{dataset}")
            kec = kecamatan_names[min(i, len(kecamatan_names)-1)]
            vegetation_df[kec] = curr
        except Exception as e:
            print(f"File vegetasi {kec} tidak dapat dimuat: {e}")
    return vegetation_df

# process data cuaca
def climate_df_processing(df):
    df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d')
    df.set_index('datetime', inplace=True)
    
    return df[(df.index.year >= start_year) & (df.index.year <= end_year)]

# process data vegetasi
def vegetation_preprocessing(df):
    df = df.iloc[:,2:]
    df = df[df.label == 'pandanwangi']
    df = df.set_index('cluster_id') 
    df = df.describe()
    df = df.T # transpose urutan waktu menjadi baris
    df = df[extract_statistics] # ambil fitur statistik yang diinginkan dari .describe()
    
    indexes = df.index
    new_indexes = []
    print(indexes)
    for i in indexes:
        date = pd.to_datetime(i, format='%Y%m%d')
        new_indexes.append(date)
    
    df['datetime'] = new_indexes
    df = df.set_index('datetime')
    df = df.asfreq(temporal_resolution, method='nearest') # resample data, dengan space temporal menjadi per 5 hari dengan metode nilai terdekat
    
    return df[(df.index.year >= start_year) & (df.index.year <= end_year)]

# function untuk mem-filter dataframe dengan metode savitzky-golay 
def smoothing_sg(df, window_length=16, polyorder=3):
    for col in ['mean', 'min', 'max']:
        filtered = savgol_filter(df[col], window_length=window_length, polyorder=polyorder)
        df[f'smoothed_{col}'] = filtered
    return df


# melakukan differencing pada data time series
def difference_df(df):
    return df.diff()
    
# mengecek stasioneritas pada data dengan Dicky-Fuller test dengan alpha 5%
def test_stationarity(df):
    results = {}
    for col in df.columns:
        adf_result = sm.tsa.stattools.adfuller(df[col].dropna())
        results[col] = {
            'ADF Statistic': adf_result[0],
            'p-value': adf_result[1],
            'Stationary': adf_result[1] < 0.05
        }
    return pd.DataFrame(results).T

# hitung korelasi dengan lag
def calculate_lagged_correlation(climate_df, vegetation_df, max_lag=30):
    vegetation_stats = statistic_corr
    
    lag_correlation = {
        'statistic': [],
        'climate_feature': [],
        'lag': [],
        'correlation': [],
        'p_value': []
    }
    
    for stat in vegetation_stats:
        if stat in vegetation_df.columns:
            veg_series = vegetation_df[stat]
            
            for feature in climate_df.columns:
                climate_series = climate_df[feature]
                
                intersecting_idx = veg_series.index.intersection(climate_series.index) # kedua dataset yang dibandingkan mempunyai index yang sama (time range yang sama)
                veg_adjusted = veg_series.loc[intersecting_idx]
                climate_adjusted = climate_series.loc[intersecting_idx]
                
                # hitung cross-correlation untuk lag 0 hingga 30 (pada time series)
                for lag in range(max_lag+1):
                    x = climate_adjusted.iloc[:-lag].values if lag != 0 else climate_adjusted.values
                    y = veg_adjusted.iloc[lag:].values if lag != 0 else veg_adjusted.values
                    
                    corr, p_value = stats.pearsonr(x, y)
                    
                    lag_correlation['statistic'].append(stat)
                    lag_correlation['climate_feature'].append(feature)
                    lag_correlation['lag'].append(lag)
                    lag_correlation['correlation'].append(corr)
                    lag_correlation['p_value'].append(p_value)
    
    return pd.DataFrame(lag_correlation)

# granger causality
def granger_causality_test(climate_df, vegetation_df, max_lag=10):
    
    results = {
        'climate_feature': [],
        'vegetation_stat': [],
        'max_lag': [],
        'min_p_value': [],
        'causality_direction': []
    }
    
    for climate_feature in climate_df.columns:
        for veg_stat in vegetation_df.columns:
            if veg_stat in statistic_corr:
                
                intersecting_idx = climate_df.index.intersection(vegetation_df.index)

                climate_series = climate_df[climate_feature].loc[intersecting_idx]
                veg_series = vegetation_df[veg_stat].loc[intersecting_idx]
                
                # data yang akan dilakukan granger test
                data = pd.DataFrame({
                    'climate': climate_series,
                    'vegetation': veg_series
                })
                
                # Test if climate Granger-causes vegetation
                try:
                    climate_to_veg = grangercausalitytests(data[['vegetation', 'climate']], 
                                                          maxlag=max_lag)

                    # print("grangertest:")
                    # print(climate_to_veg)

                    min_p_climate_to_veg = min([climate_to_veg[lag][0]['ssr_chi2test'][1] 
                                             for lag in range(1, max_lag+1)])
                    
                    # Determine causality direction
                    if min_p_climate_to_veg < 0.05:
                        direction = 'granger-causal'
                        min_p = min_p_climate_to_veg
                    else:
                        direction = 'none'
                        min_p = min(min_p_climate_to_veg, min_p_veg_to_climate)
                    
                    results['climate_feature'].append(climate_feature)
                    results['vegetation_stat'].append(veg_stat)
                    results['max_lag'].append(max_lag)
                    results['min_p_value'].append(min_p)
                    results['causality_direction'].append(direction)
                
                except Exception as e:
                    print(f"Error in Granger test: {e}")
    
    return pd.DataFrame(results)

# Visualize time series data
def plot_time_series(climate_df, vegetation_df, feature, stat):
    fig, ax1 = plt.subplots(figsize=(14, 6))
    
    # Plot climate data
    color = 'tab:blue'
    ax1.set_xlabel('Date')
    ax1.set_ylabel(feature, color=color)
    climate_series = climate_df[feature]
    ax1.plot(climate_series.index, climate_series.values, color=color, label=feature)
    ax1.tick_params(axis='y', labelcolor=color)
    
    # Create second y-axis for vegetation data
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel(f'Vegetation {stat}', color=color)
    veg_series = vegetation_df[stat]
    ax2.plot(veg_series.index, veg_series.values, color=color, label=f'Vegetation {stat}')
    ax2.tick_params(axis='y', labelcolor=color)
    
    # Add title and legend
    plt.title(f'Time Series: {feature} vs Vegetation {stat}')
    fig.tight_layout()
    fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))
    plt.savefig(f'{viz_dir}time_series_{feature}_{stat}.png')
    plt.close()

# Visualize lag correlation
def plot_lag_correlation(lag_corr_df, vegetation_stat, climate_feature):
    subset = lag_corr_df[(lag_corr_df['statistic'] == vegetation_stat) & 
                         (lag_corr_df['climate_feature'] == climate_feature)]
    
    plt.figure(figsize=(10, 5))
    plt.plot(subset['lag'], subset['correlation'], marker='o')
    plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
    
    # Add significance threshold lines (assuming p<0.05)
    significant = subset[subset['p_value'] < 0.05]
    plt.scatter(significant['lag'], significant['correlation'], color='red', 
                s=80, label='p < 0.05', zorder=3)
    
    plt.title(f'Lag Correlation: {climate_feature} vs Vegetation {vegetation_stat}')
    plt.xlabel('Lag (Negative: Climate leads, Positive: Vegetation leads)')
    plt.ylabel('Correlation Coefficient')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{viz_dir}lag_correlation_{climate_feature}_{vegetation_stat}.png')
    plt.close()

# Visualize correlation heatmap
def plot_correlation_heatmap(lag_corr_df, lag=0):
    # Filter for specific lag
    lag_subset = lag_corr_df[lag_corr_df['lag'] == lag]
    
    # Prepare data for heatmap
    pivot_data = lag_subset.pivot_table(
        index='climate_feature', 
        columns='statistic', 
        values='correlation'
    )
    
    plt.figure(figsize=(12, 8))
    
    # Create heatmap
    sns.heatmap(pivot_data, annot=True, cmap='coolwarm', center=0, 
                vmin=-1, vmax=1, fmt='.2f')
    
    plt.title(f'Correlation Heatmap (Lag = {lag})')
    plt.tight_layout()
    plt.savefig(f'{viz_dir}correlation_heatmap_lag_{lag}.png')
    plt.close()
    
    # Also create p-value heatmap
    p_value_pivot = lag_subset.pivot_table(
        index='climate_feature', 
        columns='statistic', 
        values='p_value'
    )
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(p_value_pivot < 0.05, annot=p_value_pivot, cmap='viridis', 
                fmt='.3f', cbar_kws={'label': 'p-value'})
    
    plt.title(f'P-values Heatmap (Lag = {lag})')
    plt.tight_layout()
    plt.savefig(f'{viz_dir}pvalue_heatmap_lag_{lag}.png')
    plt.close()


In [13]:
def main():
    climate_df = load_climate_data(climate_dir, era5_datasets, kecamatan_names)
    vegetation_df = load_vegetation_data(vegetation_dir, available_vegetation_datasets, kecamatan_names)
    
    preprocessed_climate = {}
    preprocessed_vegetation = {}

    # lakukan preprocess pada setiap kecamatan
    for kec in kecamatan_names:
        preprocessed_climate[kec] = climate_df_processing(climate_df[kec])
        preprocessed_vegetation[kec] = vegetation_preprocessing(vegetation_df[kec])
        preprocessed_vegetation[kec] = smoothing_sg(preprocessed_vegetation[kec])

    # print(preprocessed_vegetation['warungkondang'].value_counts())
    
    # uji stasioneritas
    for kec in kecamatan_names:
        print(f"\nStationarity Test for Climate Data ({kec}):")
        climate_stationarity = test_stationarity(preprocessed_climate[kec])
        print(climate_stationarity)
        
        print(f"\nStationarity Test for Vegetation Data ({kec}):")
        vegetation_stationarity = test_stationarity(preprocessed_vegetation[kec])
        print(vegetation_stationarity)
    
    differenced_climate = {}
    differenced_vegetation = {}
    
    for kec in kecamatan_names:
        climate_stationary = test_stationarity(preprocessed_climate[kec])
        veg_stationary = test_stationarity(preprocessed_vegetation[kec])

        differenced_climate[kec] = difference_df(preprocessed_climate[kec])
        print(f"Applied differencing to climate data for {kec}")
        differenced_vegetation[kec] = difference_df(preprocessed_vegetation[kec])
        print(f"Applied differencing to vegetation data for {kec}")
    
    # adjust data agar mempunyai time series yang sama
    adjusted_data = {}
    for kec in kecamatan_names:
        intersecting_idx = differenced_climate[kec].index.intersection(differenced_vegetation[kec].index)
        adjusted_data[kec] = {
            'climate': differenced_climate[kec].loc[intersecting_idx].interpolate().bfill().ffill(),
            'vegetation': differenced_vegetation[kec].loc[intersecting_idx].interpolate().bfill().ffill()
        }
        
    # print("adjusted_data", adjusted_data)
    # hitung korelasi dengan faktor lag hingga 30
    results = {}
    for kec in kecamatan_names:
        print(f"\nLagged correlations for {kec}...")
        lag_corr_df = calculate_lagged_correlation(
            adjusted_data[kec]['climate'], 
            adjusted_data[kec]['vegetation'], 
            max_lag=30
        )
        results[kec] = lag_corr_df

    print("hasil result:")
    print(results)
    print(selected_kec)
    # cari korelasi terkuat
    for kec in kecamatan_names:
        lag_corr = results[kec]
        # ambil korelasi signifikan dan terkuat
        significant = lag_corr[lag_corr['p_value'] < 0.05]
        strongest = significant.loc[significant['correlation'].abs().sort_values(ascending=False).index]
        
        print(f"\nTop 10 strongest correlations for {kec}:")
        print(strongest.head(15))
    
    # Granger causality test
    for kec in kecamatan_names:
        print(f"\nGranger causality test for {kec}...")
        granger_results = granger_causality_test(
            adjusted_data[kec]['climate'],
            adjusted_data[kec]['vegetation']
        )
        
        # Filter for significant causal relationships
        significant_causality = granger_results[granger_results['min_p_value'] < 0.05]
        
        print(f"\nSignificant causal relationships for {kec}:")
        print(significant_causality)
    
    # 1. Time series plots for top correlations
    if len(significant) > 0:
        top_correlation = strongest.iloc[0]
        feature = top_correlation['climate_feature']
        stat = top_correlation['statistic']

        plot_time_series(
            adjusted_data[selected_kec]['climate'],
            adjusted_data[selected_kec]['vegetation'],
            feature, stat
        )
    
    # 2. Lag correlation plots
    for feature in chirts_features[:3]:  # Plot first 3 features for example
        for stat in ['mean', 'smoothed_mean']:
            plot_lag_correlation(results[selected_kec], stat, feature)
    
    # 3. Correlation heatmaps at different lags
    for lag in [-10, -5, 0, 5, 10]:
        plot_correlation_heatmap(results[selected_kec], lag)
    
    print("\nAnalysis complete! Visualization files have been saved.")

if __name__ == "__main__":
    main()

Index(['19880414', '19880601', '19880617', '19880820', '19890706', '19900420',
       '19900607', '19900725', '19900810', '19910914',
       ...
       '20241109', '20250418', '20250420', '20250423', '20250428', '20250503',
       '20250505', '20250506', '20250508', '20250513'],
      dtype='object', length=655)

Stationarity Test for Climate Data (warungkondang):
                                      ADF Statistic   p-value Stationary
temperature_2m                             -4.92619  0.000031       True
temperature_2m_min                        -4.226481  0.000594       True
temperature_2m_max                        -4.818168   0.00005       True
soil_temperature_level_1                  -4.012183  0.001348       True
soil_temperature_level_2                  -3.425184   0.01013       True
volumetric_soil_water_layer_1             -4.945737  0.000028       True
volumetric_soil_water_layer_2             -4.202466  0.000653       True
volumetric_soil_water_layer_3             -3.7625

ValueError: zero-size array to reduction operation fmin which has no identity

<Figure size 1200x800 with 0 Axes>